As pastas e arquivos em "Compartilhado comigo" não aparecem diretamente como pastas normais no Google Drive montado no Colab.
Uma maneira de acessar essas pastas é criando um atalho para essa pasta no "Meu Drive".

Vá até a pasta "Compartilhado comigo" no seu Google Drive através do navegador.
Clique com o botão direito na pasta ou arquivo que você deseja copiar.
Selecione "Organizar" e "Adicionar atalho ao Meu Drive".
Escolha onde deseja colocar o atalho dentro do "Meu Drive".

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')


!cp -r "/content/drive/My Drive/Dataset" "/content/"



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
cp: cannot stat '/content/drive/My Drive/Dataset': No such file or directory


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
print(os.listdir('/content/drive/My Drive/Dataset'))

NameError: name 'os' is not defined

In [ ]:
import pandas as pd

# Lê o arquivo CSV
df = pd.read_csv('/content/Dataset/train.csv')

# Seleciona a coluna 'image_id'
id_column = df['image_id']

# Cria uma lista com os IDs das imagens
image_ids = id_column.tolist()

# Imprime a lista de IDs
print(image_ids)

[388811999, 613462606, 1360338805, 1672636630, 461614796, 530620473, 773597682, 1803952236, 458553546, 1434858530, 195400299, 605115808, 1967460233, 76321767, 2018825992, 797737008, 1095412840, 311168378, 1696075167, 1222121284, 1774497291, 420829283, 888426604, 1326732476, 1869230850, 1086977375, 1417771843, 1882170663, 1926447510, 1820235658, 1915182298, 1096308702, 1810004579, 199041775, 671458270, 1069030033, 1812305728, 1007401721, 1017363921, 457443056, 1767654639, 1203123196, 1652537920, 1827516400, 91797551, 100147863, 740439778, 1429876928, 1127855363, 2025746473, 1995339680, 2026358482, 982883275, 1420153030, 1225478313, 1717036504, 1446339465, 1563019139, 212923136, 220808925, 382701440, 1852393673, 156027166, 1289574047, 1297379916, 1435663649, 610792199, 835777184, 551129512, 1191890529, 345896545, 739781139, 234842188, 238763713, 739383742, 1222788247, 1789731455, 2024785721, 1187678911, 1360800566, 1949261883, 338330540, 1475901500, 740921229, 834999110, 1210098281, 1928

In [ ]:
import os
import pandas as pd

# Define o diretório do dataset
# alterar a string com o nome do diretório onde se encontram as imagens do dataset
dataset_dir = '/content/Dataset/train_images'

# Lê o arquivo CSV
df = pd.read_csv('/content/Dataset/train.csv')

# Itera sobre os arquivos no diretório
for filename in os.listdir(dataset_dir):
    # Verifica se o arquivo é um arquivo PNG
    if filename.endswith(".png"):
        # Divide o nome do arquivo em partes usando '@' como separador
        parts = filename.split('@')
        # Pega a parte que vem depois do '@'
        part_after_at = parts[1]
        # Remove a extensão '.png'
        image_id = part_after_at[:-4]
        # Adiciona o ID da imagem à lista
        image_ids.append(image_id)

# Compara os IDs das imagens com o 'image_id' do CSV
for image_id in image_ids:
    # Verifica se o ID da imagem está presente no CSV
    if image_id in df['image_id'].values:
        print(f"O ID da imagem '{image_id}' está presente no CSV.")
